In [ ]:
#!conda activate py38_estnltk1.7
#hpc serveris peaks sobima estnltk_collocations_py38
#!conda install --channel conda-forge pygraphviz
#!conda install -c conda-forge ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

# Verb - compound - obl - obl_case kollokatsioonide kogumine koondkorpusest


# v12
Kollokatsiooniandmete salvestamisel tabelisse jäeti välja kollokatsioonid, milles:

* verb oli umbisikuline (<code>feats</code> sisaldas parameetrit <code>imps</code>);
* verbi aeg polnud ükski järgnevatest: <code>past</code>, <code>impf</code>, <code>pres</code> (<code>feats</code> ei sisaldanud parameetreid <code>past</code> ega <code>impf</code> ega  <code>pres</code>).

### Tabelid

### TABEL1 
**verb\_compound\_obl**

Paksu kirjaga on märgitud väljad, mis moodustavad unikaalse indeksi.

| väli | tüüp  |  kirjeldus | näide | märkus |
| --- | --- | --- | --- | --- |
| id | int | kollokatsiooni<br/>unikaalne ID| *56* | |
| **verb** | text | tegusõna lemma | *kirjutama* | |
| **verb_compound** | text | määrsõna(de) lemma(d) | *alla,kokku* | mitme määrsõna korral on lemmade eraldajaks koma ja lemmad on alfabeetilises järjestuses |
| **obl_root** | text| täiendi juure lemma nn "puhastatud kuju"| *kurjajuur* | |
| **obl\_root\_compound** | text| täiendi juure lemma liitsõnamärkidega*| *kurja_juur* | |
| **obl_case** | text | täiendi juure kääne | *ad* | vt [Käänded](#käänded) |
| **obl_k** | text | täiendi juurele alluv kaassõna lemma  | *peale* | tipp, mille <code>deprel=case</code> ja <code>pos=K</code>,<br/>mitme määrsõna korral on lemmade eraldajaks koma, lemmad on alfabeetilises järjestuses |
| **ner_loc** | text | täiendi fraasi asetus NER 'LOC' spani suhtes  | *intersects* | vt [OBL asetus](#obl_asetus) |
| **ner_per** | text | --\|\|-- 'PER' spani suhtes | *match* | vt [OBL asetus](#obl_asetus) |
| **ner_org** | text | --\|\|-- 'ORG' spani suhtes | *contains* | vt [OBL asetus](#obl_asetus) |
| **timex** | text |  --\|\|-- TIMEX spani suhtes | *-* | vt [OBL asetus](#obl_asetus)|
| count | int | kollokatsiooni esinemiste arv üle korpuse | *56* |  | 
| phrase_type | text | täiendi fraasi tüüp | *null* |   täitmata |

### TABEL2 

**verb\_compound\_obl\_examples**


| väli | tüüp  |  kirjeldus |	näide | märkus |
| --- | --- | --- | --- | --- |
| **row_id** | int | rea id tabelis TABEL1| | |
| **sentence_id** | int | kollektsiooni id postgres andmebaasis | | |
| **root_id** | int | OBL fraasi juur (sõna nr) | | |
| count | int | mitu korda näide esines |  | täitsa võimalik, et selle välja väärtus on antud versiooni tabelis alati 1 |




### OBL asetus
| väärtus | kirjeldus  |  
| --- | --- |
| **-** |           OBL ei ole kautud ühegi spaniga|
| **match** |      OBL span langeb kokku NER/TIMEX spaniga|
| **contains** |    OBL spani sees on NER/TIMEX span|
| **inside** |        OBL span on  NER/TIMEX spani sees|
| **intersects** |  OBL span lõikub NER/TIMEX spaniga|



### Käänded
| kääne|  nimetus| 
| --- | --- |
| nom |  nimetav | 
| gen | omastav | 
| part | osastav | 
| adit | lühike sisseütlev | 
| ill | sisseütlev | 
| in |  seesütlev | 
| el |  seestütlev | 
| all |  alaleütlev |
| ad |  alalütlev | 
| abl |  alaltütlev | 
| tr | saav | 
| term |  rajav | 
| es |  olev | 
| abes |  ilma | 
| kom | kaasa | 
| \<käändumatu\> | täiendil puudus kääne | 





In [ ]:
%load_ext autoreload
%autoreload 2

from estnltk.storage.postgres import LayerQuery

from datetime import datetime
# functions for creating database and collecting collocations
from collect_functions import *

from data_helpers.db_reader import DbReader

collection_name = 'koondkorpus_sentences' 
# collection_name = 'koondkorpus_sentences_test_5000_sg_thread'


TYPE = 'verb_compound_obl'
TABLENAME = f'{TYPE}'
BATCH_SIZE = 50000

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
db_file_name = f"v12_{collection_name}_{TYPE}_collocations_{date_time}.db"
my_sqlite_db = DbMethods(db_file_name=db_file_name, table1_name=TYPE, table2_name=TYPE+'_examples')
my_sqlite_db.prep_coll_db()


my_db_reader = DbReader(pgpass_file='~/.pgpass',\
                          schema='estonian_text_corpora',\
                          role='estonian_text_corpora_read',\
                          temporary=False,\
                          collection_name=collection_name)
my_db_reader.set_layers(['v171_named_entities', 'v172_stanza_syntax', 'v172_obl_phrases', 'v172_pre_timexes'])

# vaatame ainult neid lauseid, kus on mõni tegusõna ka
# NB! kas saab olla obl, kus ülemus ei ole tegusõna
additional_filters = [LayerQuery('morph_analysis', partofspeech='V')]
collocations = {}
for collection_id, text in my_db_reader.get_collections(shuffle=False, queries=additional_filters, progressbar='ascii'): 
    collocations, = extract_something(text, collection_id, collocations )
    if not collection_id == 0 and not collection_id % BATCH_SIZE:
        my_sqlite_db.save_coll_to_db(collocations, collection_id)
        collocations = {}

# saving last batch
my_sqlite_db.save_coll_to_db(collocations, collection_id)
my_sqlite_db.index_fields()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO:storage.py:41: connecting to host: 'postgres.keeleressursid.ee', port: '5432', dbname: 'estonian-text-corpora', user: 'zummy'
INFO:storage.py:58: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


In [ ]:
# display(text['morph_analysis'])

In [ ]:
# display(text['v172_stanza_syntax'])